In [1]:
from src.WikidataTextifier import WikidataEntity

earth = WikidataEntity.from_id("Q2")
print(earth.to_triplet())

Earth (Q2): lowest point (P1589): Challenger Deep (Q459173) | elevation above sea level (P2044): -10994 metre)
Earth (Q2): shape (P1419): geoid (Q185969) | statement is subject of (P805): figure of the Earth (Q437882))
Earth (Q2): has part(s) (P527): Earth's crust (Q15316)
Earth (Q2): has part(s) (P527): Earth's core (Q193927)
Earth (Q2): has part(s) (P527): mantle (Q101949)
Earth (Q2): has part(s) (P527): atmosphere of Earth (Q3230)
Earth (Q2): has part(s) (P527): Earth's surface (Q1349417)
Earth (Q2): has part(s) (P527): biosphere (Q42762)
Earth (Q2): has part(s) (P527): hydrosphere (Q83169)
Earth (Q2): has part(s) (P527): lithosphere (Q83296)
Earth (Q2): has part(s) (P527): Earth's magnetic field (Q6500960)
Earth (Q2): has part(s) (P527): continent (Q5107)
Earth (Q2): has part(s) (P527): hemisphere of the Earth (Q399984)
Earth (Q2): has part(s) (P527): Europe (Q46)
Earth (Q2): has part(s) (P527): Asia (Q48)
Earth (Q2): has part(s) (P527): Africa (Q15)
Earth (Q2): has part(s) (P527):